In [ ]:
pip install -q transformers[torch] datasets accelerate seqeval evaluate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")


In [ ]:
from datasets import load_dataset

model_name = "xlm-roberta-base"
panx_tr = load_dataset("xtreme", name="PAN-X.tr")

PAN-X.tr/train-00000-of-00001.parquet:   0%|          | 0.00/994k [00:00<?, ?B/s]

PAN-X.tr/validation-00000-of-00001.parqu(…):   0%|          | 0.00/503k [00:00<?, ?B/s]

PAN-X.tr/test-00000-of-00001.parquet:   0%|          | 0.00/504k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
panx_tr

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

In [ ]:
tags = panx_tr["train"].features["ner_tags"].feature
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'])

In [ ]:
index2tag = {id: tag for id, tag in enumerate(tags.names)}
tag2index = {tag: id for id, tag in enumerate(tags.names)}

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
# TOKEN İLE SAYILARA ÇEVRİLİYOR VE BÖLÜNÜYOR EĞER ÖNCEKİ KELİMENİN DEVAMIYSA VEYA SATIR BAŞI VE SONUYSA -100 KOYUYORUZ.
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
  labels = []
  for idx, label_example in enumerate(examples["ner_tags"]):
    word_ids = tokenized_inputs.word_ids(batch_index=idx)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100)
      else:
        label_ids.append(label_example[word_idx])
      previous_word_idx = word_idx
    labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [ ]:
tokenized_de = panx_tr.map(tokenize_and_align_labels, batched=True, remove_columns=panx_de["train"].column_names)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
#Modelin ürettiği sayısal tahminlerden en yüksek olasılığı seçip, -100 olan gereksiz parçaları (ekler ve özel karakterler) temizleyerek bunları "PER", "LOC" gibi anlamlı etiketlere dönüştüren ve başarı ölçümü için temiz listeler hazırlayan fonksiyondur.
def align_predictions(predictions, label_ids):
  preds = np.argmax(predictions, axis=2)
  batch_size, seq_len = preds.shape
  labels_list, preds_list = [],[]
  for batch_idx in range(batch_size):
    example_labels, example_preds = [],[]
    for seq_idx in range(seq_len):
      if label_ids[batch_idx, seq_idx] != -100:
        example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
        example_preds.append(index2tag[preds[batch_idx][seq_idx]])
    labels_list.append(example_labels)
    preds_list.append(example_preds)
  return preds_list, labels_list

In [ ]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
  y_pred, y_true = align_predictions(eval_pred.predictions, eval_pred.label_ids)
  return {"f1":f1_score(y_true,y_pred)}

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
def model_init():
  return AutoModelForTokenClassification.from_pretrained(
      model_name,
      num_labels=len(tags.names),
      id2label=index2tag,
      label2id=tag2index
  ).to(device)

In [ ]:
training_args = TrainingArguments(
    output_dir = "xlm_roberta_ner_musa",
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    eval_strategy="epoch",
    weight_decay = 0.01,
    push_to_hub = True,
    report_to = "none"
)


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
trainer = Trainer(
    model_init = model_init,
    args = training_args,
    data_collator = DataCollatorForTokenClassification(tokenizer),
    compute_metrics = compute_metrics,
    train_dataset = tokenized_de["train"],
    eval_dataset = tokenized_de["validation"]
)

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

XLMRobertaForTokenClassification LOAD REPORT from: xlm-roberta-base
Key                         | Status     | 
----------------------------+------------+-
lm_head.layer_norm.bias     | UNEXPECTED | 
lm_head.layer_norm.weight   | UNEXPECTED | 
lm_head.dense.weight        | UNEXPECTED | 
roberta.pooler.dense.weight | UNEXPECTED | 
roberta.pooler.dense.bias   | UNEXPECTED | 
lm_head.bias                | UNEXPECTED | 
lm_head.dense.bias          | UNEXPECTED | 
classifier.bias             | MISSING    | 
classifier.weight           | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
trainer.train()

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

XLMRobertaForTokenClassification LOAD REPORT from: xlm-roberta-base
Key                         | Status     | 
----------------------------+------------+-
lm_head.layer_norm.bias     | UNEXPECTED | 
lm_head.layer_norm.weight   | UNEXPECTED | 
lm_head.dense.weight        | UNEXPECTED | 
roberta.pooler.dense.weight | UNEXPECTED | 
roberta.pooler.dense.bias   | UNEXPECTED | 
lm_head.bias                | UNEXPECTED | 
lm_head.dense.bias          | UNEXPECTED | 
classifier.bias             | MISSING    | 
classifier.weight           | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Epoch,Training Loss,Validation Loss,F1
1,0.402209,0.143976,0.883404
2,0.277428,0.140479,0.912750


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss,F1
1,0.402209,0.143976,0.883404
2,0.277428,0.140479,0.912750


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
trainer.push_to_hub(commit_message="Upload trained XLM-RoBERTa model and tokenizer")
tokenizer.push_to_hub("xlm-roberta-ner-musa")

In [ ]:
test_text = "Musa şu anda İstanbuldaki'deki evinde oturuyor."

In [ ]:
def predict(text):
  inputs = tokenizer(text,return_tensors="pt").to(device)
  with torch.no_grad():
    logits = trainer.model(**inputs).logits
  predictions = torch.argmax(logits, dim=2)
  tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
  preds = [index2tag[p.item()] for p in predictions[0]]
  return list(zip(tokens, preds))

In [ ]:
print(predict(test_text))